# Lambda Athena Lab

### Introduction

In this lesson, let's work with a lambda function that will use Athena to query S3 when a csv file is uploaded to the relevant bucket.

<img src="./athena-workflow.png" width="60%">

### Where we left off

It probably makes sense to use some of our existing work in building our data pipeline.

For example, remember in our [S3 to Athena](https://colab.research.google.com/github/data-engineering-jigsaw/s3-to-athena/blob/main/lesson/index.ipynb) lesson, that we already set up a glue crawler that would crawl our specified s3 bucket, and set up an IAM role so that glue had access to this s3 bucket.

<img src="./crawler.png">

Let's begin by making sure our Athena connection to s3 still works.

In our code [located here](https://github.com/data-engineering-jigsaw/lambda-athena-lab.git), if you open the `console.py` file, with some setup, you should be able to run the code at the bottom of the file.

```python
lambda_handler({}, {})
```

> **To do so**: you'll have to change the `output_bucket_name` to the bucket where you store the query results, and the `db_name`.  Then, when executing the query below, you'll have to change `jigsawtexasquery` to the bucket where your input data is stored.

### Noticing our updates

Before moving on, let's just note a couple of changes that we made in the code since our last codebase.

The main change is just that we wrapped our code in a function named `lambda_handler`, that takes arguments of `event` and `context`.

We also made some changes that will help us with logging.  For example, at the top of the function, we print out when our function was called.  And we also print out the `results_df`.  

Finally, instead of just returning the resulting dataframe, we convert the dataframe to a list of dictionaries, as lambda requires us to return a datatype that it can convert to JSON.

* Changing the query

Ok, now let's change the query a little.  For example, let's say that we only want to retrieve the records where the `total_receipts` is over 5000.

```python
query = "SELECT * FROM jigsawtexasquery where total_receipts > 5000"
```

### Adding our lambda function

Now we want to do the following. We want to set up a lambda function, so that when we drag and drop a csv file into our query bucket, the lambda function will automatically run our athena query which will place the records with `total_receipts` over 5000 into our output bucket.

Essentially, we'll need to set up the lambda function to call our athena code when an S3 file is uploadded to our bucket.

To do this:

1. Go through the steps of creating a lambda function that has access to S3.  Look at the [following documentation](https://docs.aws.amazon.com/lambda/latest/dg/with-s3-example.html) for reference.

2. Then just make sure the s3 trigger is set up properly.  So upload something to the s3 relevant bucket, and use cloudwatch to check the logs and confirm that our lambda function was invoked.

3. Then upload the code that queries athena to the lambda function.  And make sure it works by triggering the lambda function by sending a test event to the function.  

> This will break.

There are a couple of errors you may run into at this point.

* Error: `No module named lambda function`

<img src="./no-module-error.png" width="100%">

If you changed the name of the file `lambda_function` to `console.py`, then lambda will throw an error.  This because it's looking for a file `lambda_function`, and a function inside of it called `lambda_handler` -- if it doesn't find both of those, it throws an error.

If you scroll down in the `code` panel, you can see where this is configured, under `Runtime settings`, where the Handler is specified.

<img src="./runtime-settings.png">

So make sure you have a file called `lambda_function` and that the file has a function called `lambda_handler` that you expect to be invoked.

* Another error: `No module named pandas`

<img src="./no-module-pandas.png">

Even if this is set up properly, you'll run into another error explaining that pandas is not installed.  The issue here is that the environment our lambda is running on does not have pandas.  
To have pandas installed in this environment we add a layer to our lambda environment.  There's an [excellent tutorial](linkedin.com/pulse/add-external-python-libraries-aws-lambda-using-layers-gabe-olokun/) explaining how to install a custom layer - so that we can add whatever libraries we want to our environment.  However, in this case, a prebuilt AWS layer will work just fine.

To add one of AWS's prebuilt layers, from the lambda console, go to the `code` tab (the first one), and scroll down.  There, you'll see an option to `Add a layer`.

<img src="./add-layer.png">

So click on `Add a layer`, and keep `Aws layers` selected, and from there select the `AWSSDKPandas-Python310` layer.

<img src="./pandas-layer.png">

Set the version to the most recent  available version, and then click `Add`.  If you look at the panel for your lambda function, you should see the number of layers updated to 1.

<img src="./lambda-layer.png">

Ok, so try to run the test again.  It probably won't work, but we're getting closer.

* Another error

<img src="./start-query-error.png">

This time, it looks like our lambda function does not have the correct permission to execute athena.  If we look at this [blog post](https://simplemaps.com/resources/athena-over-url), we can get a sense of how to fix this.  We likely need to modify the permissions listed there, to include both StartQueryExecution and GetQueryExecution.

Ok, so let's add permission to Athena by going to our lambda function's configuration, and then click on the role.

<img src="./config-role.png">

From there, add a policy that gives access to athena, glue, and the s3 input and output buckets (if permission is not already there for them), and cloudwatch (if this permission is not already included).

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "athena:*",
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": "s3:*",
            "Resource": [
                "arn:aws:s3:::jigsawtexasresults/*",
                "arn:aws:s3:::jigsawtexasresults",
                "arn:aws:s3:::jigsawtexasquery",
                "arn:aws:s3:::jigsawtexasquery/*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": "glue:*",
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:PutLogEvents",
                "logs:CreateLogGroup",
                "logs:CreateLogStream"
            ],
            "Resource": "arn:aws:logs:*:*:*"
        }
    ]
}
```

* If all else fails

If you keep getting permission errors, you can just give administrator access to the lambda function and go from there.

<img src="./admin-access.png">

* (Another error maybe) a timeout error

<img src="./timeout-error.png" width="70%">

At this point you may get a timeout error.  [Google this error](https://stackoverflow.com/questions/62948910/aws-lambda-errormessage-task-timed-out-after-3-00-seconds) and see how to fix it.

If you are able to get the test event to work move onto the next step.

4. Upload the csv file that is provided to the s3 bucket and confirm that the query results are in the results bucket.

### Triggering the lambda from boto

So now that we uploaded a file, and seen our lambda function called, which executes athena.  The next step is to use boto to make a request.  

<img src="./updated-pipeline.png" width="80%">

This should make a request to the external api, and upload a new object to the query bucket.  Then the new object in the query bucket should call the lambda function, which will have athena query the bucket and place the matching results in our output bucket.

We can kick this off by navigating to the `extract_load` folder.  And from there, you can update the `query_bucket_name` to be your query bucket.  And then call the `upload_console.py` file.  

Confirm this occurred with the cloudwatch logs.

### Summary

In this lesson, we set up a pipeline so that when we upload a file to S3, our lambda function triggers athena.

### What's next

One thing to improve from the above is that when we make a request to the API, we then query our entire bucket to find the selected queries.  

It would be better if we only select the recently updated queries.  One way to accomplish this would be, when querying the API to store the results in a dated folder (for example receipts/may_31_2023).  Glue will turn these folders into a different column values for a column called partition.  And in our query we can select for those records in the current dates folder that also meet any additional criteria (eg. having total receipts over 5000).

```python
from datetime import date
today = date.today().strftime("%b_%d_%Y")
query_results(f"SELECT * FROM jigsawtexasquery where total_receipts > 5000 and partition = {today}")
```

If you're feeling frisky try to get our code to only search through the recently updated files.  And from there, you may want to look into [adding a custom layer](https://www.linkedin.com/pulse/add-external-python-libraries-aws-lambda-using-layers-gabe-olokun/) instead of using our pre-built one.


### Resources

[Adding a custom layer](https://www.linkedin.com/pulse/add-external-python-libraries-aws-lambda-using-layers-gabe-olokun/)

[Athena Permissions](https://simplemaps.com/resources/athena-over-url)

[Another Athena Permissions Example](https://github.com/ShivakumarRavi/AWS-fetch_athena_data_from_lambda/tree/main)

[AWS EFS vs EBS](https://stackoverflow.com/questions/29575877/aws-efs-vs-ebs-vs-s3-differences-when-to-use)